# Set up notebook

In [1]:
MY_API_KEY='7ca7d761c1c9d2b36a6b6d0857c78344f9b7f8b0'

In [2]:
!pip install censusdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import censusdata
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def add_FIP(df):
    county_fip=[]
    tract_fip=[]
    block_fip=[]
    
    for county in df.index:
        county_=county.geo[0][1]+county.geo[1][1]
        tract_=county.geo[0][1]+county.geo[1][1]+county.geo[2][1]
        try:
            block_=county.geo[0][1]+county.geo[1][1]+county.geo[2][1]+county.geo[3][1]
            block_fip.append(block_)
            is_block=True
        except:
            is_block=False
        
        
        county_fip.append(county_)
        tract_fip.append(tract_)
        
        
        #place_name.append(county.name)
    df['County'] =county_fip
    df['Tract'] =tract_fip
    if is_block: 
        df['block_FIP'] =block_fip
    #df['place']=place_name
    return df

In [5]:
year=2019 #2019 
county_list=['005', '047', '081',  '061'] #bx, bk, manhattan, queens

#Race and population

In [6]:
#Hsu designates five groups: Asian, Black, Hispanic, White, and no majority groups.
censusdata.printtable(censusdata.censustable('acs5', year, 'B03002'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B03002_001E  | HISPANIC OR LATINO ORIGIN BY R | !! Estimate Total:                                       | int  
B03002_002E  | HISPANIC OR LATINO ORIGIN BY R | !! !! Estimate Total: Not Hispanic or Latino:            | int  
B03002_003E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: White a | int  
B03002_004E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Black o | int  
B03002_005E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: America | int  
B03002_006E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Asian a | int  
B03002_007E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Lati

In [7]:
race_variables={'B03002_001E':'Race_Total','B03002_003E':'White','B03002_004E':'Black','B03002_012E':'Hispanic_Latino',
               'B03002_006E':"Asian", 'B03002_009E':'Mixed'}
#total-white-black-hispanic-asian-mixed=minor races

In [8]:
def make_race_df(county_code):
    race_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(race_variables.keys()),key=MY_API_KEY)
    race_data.columns=race_data.columns.map(race_variables)
    
    race_data=add_FIP(race_data)
    return race_data

# HH income

In [9]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B19013'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B19013_001E  | MEDIAN HOUSEHOLD INCOME IN THE | !! Estimate Median household income in the past 12 month | int  
-------------------------------------------------------------------------------------------------------------------


In [10]:
def make_income_df(county_code):
    income_data=censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       ['B19013_001E'],key=MY_API_KEY)
    income_data.columns=['Median_Income']
    income_data=add_FIP(income_data)
    

    return income_data

#Education attainment

In [11]:
censusdata.printtable(censusdata.censustable('acs5', year, 'B15003'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B15003_001E  | EDUCATIONAL ATTAINMENT FOR THE | !! Estimate Total:                                       | int  
B15003_002E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total: No schooling completed             | int  
B15003_003E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total: Nursery school                     | int  
B15003_004E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total: Kindergarten                       | int  
B15003_005E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total: 1st grade                          | int  
B15003_006E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total: 2nd grade                          | int  
B15003_007E  | EDUCATIONAL ATTAINMENT FOR THE | !! !! Estimate Total: 3rd grade              

In [12]:
#bachelor's and above only
edu_variables = {'B15003_001E':'Edu_Tot','B15003_022E':'Bachelor', 'B15003_023E':'Master', 
                 'B15003_024E':'Prefessional', 'B15003_025E':'Doctorate', 'B03002_001E':'Pop_Total'}

In [13]:
def make_edu_df(county_code):
    edu_data = censusdata.download('acs5', year, censusdata.censusgeo([('state', '36'), ('county', county_code), ('tract', '*')]),
                                       list(edu_variables.keys()),key=MY_API_KEY)
    edu_data.columns=edu_data.columns.map(edu_variables)
    
    edu_data=add_FIP(edu_data)
    return edu_data

#Combine to one dataframe

In [14]:
race = []
income = []
edu = []

for county in county_list:
    race.append(make_race_df(county))
    income.append(make_income_df(county))
    edu.append(make_edu_df(county))

##Dataframe for race

The race categories here included are: 

*   White
*   Black
*   Hispanic or Latino
*   Asian
*   Other races
*   Mixed



In [15]:
race_col = ['Race_Total','White','Black',
            'Hispanic_Latino',"Asian",'Mixed','County', 'Tract']

race_d = pd.DataFrame()
race_df = pd.concat([race_d, (pd.DataFrame(data=race[0], columns=race_col)),
                     (pd.DataFrame(data=race[1], columns=race_col)),
                     (pd.DataFrame(data=race[2], columns=race_col)),
                     (pd.DataFrame(data=race[3], columns=race_col))]).reset_index(drop=True)

#total-white-black-hispanic-asian-mixed=minor races
race_df['Other'] = race_df['Race_Total']-race_df.iloc[:,1:-2].sum(axis=1)

#% of each race
races = ['White', 'Black',
        'Hispanic_Latino',"Asian",
        'Other', 'Mixed']
for i in range(len(races)):
  race_df['%{}'.format(races[i])]=(race_df['{}'.format(races[i])]/race_df['Race_Total'])*100

race_df = race_df.iloc[:, [6,7,9,10,11,12,13,14]]

race_df.head(2)

,County,Tract,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed
0,36005,36005020000,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707
1,36005,36005020501,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032


## Dataframe for income

In [16]:
inc_col = ['Median_Income','County', 'Tract']


inc_d = pd.DataFrame()
inc_df = pd.concat([inc_d, (pd.DataFrame(data=income[0], columns=inc_col)),
                     (pd.DataFrame(data=income[1], columns=inc_col)),
                     (pd.DataFrame(data=income[2], columns=inc_col)),
                     (pd.DataFrame(data=income[3], columns=inc_col))]).reset_index(drop=True)

inc_df.head(2)

,Median_Income,County,Tract
0,40184,36005,36005020000
1,17601,36005,36005020501


## Dataframe for education attainment

The education attainment is reported for the population above 25 years old. The percentage calculated here is $\frac{Bachelor's\: or\: higher}{Total\:population \:(all\:ages)}$

In [17]:
edu_col = ['Edu_Tot', 'Bachelor','Master', 
           'Prefessional', 'Doctorate','Pop_Total','County', 'Tract'] 

edu_d = pd.DataFrame()
edu_df = pd.concat([edu_d, (pd.DataFrame(data=edu[0], columns=edu_col)),
                     (pd.DataFrame(data=edu[1], columns=edu_col)),
                     (pd.DataFrame(data=edu[2], columns=edu_col)),
                     (pd.DataFrame(data=edu[3], columns=edu_col))]).reset_index(drop=True)

edu_df['%Edu_Att'] = ((edu_df.iloc[:, 1:5].sum(axis=1))/edu_df['Pop_Total'])*100
edu_df = edu_df.iloc[:, -4:]

edu_df.head(2)

,Pop_Total,County,Tract,%Edu_Att
0,4711,36005,36005020000,15.516875
1,6976,36005,36005020501,7.124427


##Combine the three dataframes

In [18]:
merge1 = race_df.merge(inc_df,how='inner',on='Tract')
merge1 = merge1.iloc[:, 1:-1]
merge1.head(2)

,Tract,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,Median_Income
0,36005020000,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707,40184
1,36005020501,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032,17601


In [19]:
df_combined = merge1.merge(edu_df,how='inner',on='Tract')
df_combined = df_combined[['Tract', 'Pop_Total', '%White', '%Black',
                '%Hispanic_Latino',"%Asian",
                '%Other', '%Mixed', 'Median_Income', '%Edu_Att']]
df_combined

,Tract,Pop_Total,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,Median_Income,%Edu_Att
0,36005020000,4711,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707,40184,15.516875
1,36005020501,6976,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032,17601,7.124427
2,36005020502,2160,1.666667,26.435185,71.481481,0.138889,0.000000,0.277778,18919,13.703704
3,36005020900,4287,3.825519,37.602053,57.872638,0.256590,0.209937,0.233263,31190,10.566830
4,36005021001,8930,2.183651,50.470325,36.125420,8.801792,1.209406,1.209406,54076,18.969765
...,...,...,...,...,...,...,...,...,...,...
2052,36061021800,6236,20.525978,58.611289,18.136626,1.860167,0.545221,0.320718,49482,30.596536
2053,36061028300,7949,19.335766,5.535287,68.964650,4.981759,0.314505,0.868034,56855,28.053843
2054,36061029100,12165,6.321414,5.318537,82.836005,4.069051,0.139745,1.315249,47054,15.330867
2055,36061029500,8182,34.808115,2.994378,56.318748,3.312149,0.378880,2.187729,73514,38.926913


In [20]:
df_combined.to_csv('census_tract_level.csv',index=False)

#Create centroid in tract shapefile and join to the df above

In [21]:
# Install Geopandas
!pip install geopandas
!pip install fiona
!pip install gdal
# Install rtree - Geopandas requirment
!pip install rtree 
# Install descartes - Geopandas requirment
!pip install descartes 
# Install plotlyExpress
!pip install plotly_express
# Install shapely for geometry
!pip install shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import geopandas as gpd

In [23]:
nys_tract = gpd.read_file('/content/tl_2019_36_tract.shp').set_crs(4326, allow_override=True)
nys_tract.shape

(4918, 13)

In [24]:
nys_tract.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Use internal point in the census tract shapefile:
*Internal point—The Census Bureau calculates an internal point (latitude and longitude coordinates) for each geographic area. For many geographic areas, the internal point is the centroid, the geographic center of the entity. For some irregularly shaped areas (such as those shaped like a crescent), the centroid may be located outside the boundaries of the entity. In such instances, the internal point is identified as a point inside the entity boundaries nearest to the centroid and, if possible, a point that is on land area, not water.*

In [25]:
nys_tract.head(2)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,081,044800,36081044800,448,Census Tract 448,G5020,S,208002,0,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712..."
1,36,081,045800,36081045800,458,Census Tract 458,G5020,S,245281,0,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713..."


In [26]:
nys_tract = nys_tract.iloc[:, [3, 10, 11, 12]]
nys_tract.head(5)

,GEOID,INTPTLAT,INTPTLON,geometry
0,36081044800,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712..."
1,36081045800,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713..."
2,36081046200,+40.7098547,-073.7879749,"POLYGON ((-73.79203 40.71107, -73.79101 40.711..."
3,36081046300,+40.7440007,-073.8710900,"POLYGON ((-73.87468 40.74335, -73.87423 40.743..."
4,36081046400,+40.7168637,-073.7869958,"POLYGON ((-73.79187 40.71379, -73.79085 40.714..."


In [27]:
tract_df = pd.DataFrame(nys_tract)
tract_df.head(5)

,GEOID,INTPTLAT,INTPTLON,geometry
0,36081044800,+40.7110219,-073.8026344,"POLYGON ((-73.80646 40.71206, -73.80556 40.712..."
1,36081045800,+40.7152626,-073.7909261,"POLYGON ((-73.79364 40.71382, -73.79362 40.713..."
2,36081046200,+40.7098547,-073.7879749,"POLYGON ((-73.79203 40.71107, -73.79101 40.711..."
3,36081046300,+40.7440007,-073.8710900,"POLYGON ((-73.87468 40.74335, -73.87423 40.743..."
4,36081046400,+40.7168637,-073.7869958,"POLYGON ((-73.79187 40.71379, -73.79085 40.714..."


In [28]:
#merge with the census df from before
tract_merged = df_combined.merge(tract_df,how='inner',left_on='Tract', right_on='GEOID')\
                          .drop(columns=['GEOID'])
tract_merged.head(2)

,Tract,Pop_Total,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,Median_Income,%Edu_Att,INTPTLAT,INTPTLON,geometry
0,36005020000,4711,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707,40184,15.516875,+40.8428349,-073.8455162,"POLYGON ((-73.84999 40.84285, -73.84939 40.842..."
1,36005020501,6976,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032,17601,7.124427,+40.8474409,-073.9236597,"POLYGON ((-73.92815 40.84551, -73.92811 40.845..."


In [29]:
#create geodataframe to save as shapefile
tract_merged_gdf = gpd.GeoDataFrame(tract_merged, geometry='geometry')
tract_merged_gdf.head(2)

,Tract,Pop_Total,%White,%Black,%Hispanic_Latino,%Asian,%Other,%Mixed,Median_Income,%Edu_Att,INTPTLAT,INTPTLON,geometry
0,36005020000,4711,2.759499,9.382297,61.197198,23.392061,2.080238,1.188707,40184,15.516875,+40.8428349,-073.8455162,"POLYGON ((-73.84999 40.84285, -73.84939 40.842..."
1,36005020501,6976,1.046445,26.333142,71.530963,0.788417,0.000000,0.301032,17601,7.124427,+40.8474409,-073.9236597,"POLYGON ((-73.92815 40.84551, -73.92811 40.845..."


In [31]:
tract_merged_gdf.to_file('tract_with_info.shp')  

<ipython-input-31-3fa75e98851d>:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  tract_merged_gdf.to_file('tract_with_info.shp')
